In [1]:
%run dados.ipynb

In [2]:
spark = (SparkSession.builder
        .master("local")
        .appName("projetosoulcode")
        .config('spark.jars','gcs-connector-hadoop3-latest.jar' )
        .config('spark.hadoop.fs.gs.impl','com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
        .config('spark.hadoop.fs.gs.auth.service.account.enable', 'true')
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile",'soulcodeproject-44ce61622773.json')
        .config('spark.hadoop.fs.gs.project.id','soulcodeproject')
        .getOrCreate())
spark.conf.set("spark.sql.caseSensitive", "true")

22/03/18 21:22:04 WARN Utils: Your hostname, eveline-Inspiron-5557 resolves to a loopback address: 127.0.1.1; using 192.168.1.47 instead (on interface enp3s0)
22/03/18 21:22:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/18 21:22:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
schema = (
    StructType([
        StructField("ID",IntegerType(), False),
        StructField("AnoNascimento", IntegerType(), False),
        StructField("GrauInstrução", StringType(), True),
        StructField("EstadoCivil", StringType(), True),
        StructField("Renda", DoubleType(), True),
        StructField("NumCrianca", IntegerType(), True),
        StructField("NumAdolescente", IntegerType(), True),
        StructField("DataConta", StringType(), True),
        StructField("UltimaCompra", IntegerType(), True),
        StructField("QtdVinho", IntegerType(), True),
        StructField("QtdFrutas", IntegerType(), True),
        StructField("QtdCarne", IntegerType(), True),
        StructField("QtdPeixe", IntegerType(), True),
        StructField("QtdDoces", IntegerType(), True),
        StructField("QtdOuro", IntegerType(), True),
        StructField("QtdComprasDesconto", IntegerType(), True),
        StructField("QtdComprasWeb", IntegerType(), True),
        StructField("QtdCompraCatalogo", IntegerType(), True),
        StructField("QtdCompraLoja", IntegerType(), True),
        StructField("NumVisitasSite", IntegerType(), True),
        StructField("AceitouCam3", IntegerType(), True),
        StructField("AceitouCam4", IntegerType(), True),
        StructField("AceitouCam5", IntegerType(), True),
        StructField("AceitouCam1", IntegerType(), True),
        StructField("AceitouCam2", IntegerType(), True),
        StructField("Reclamacao", IntegerType(), True),
        StructField("CustoContato", IntegerType(), True),
        StructField("Receita", IntegerType(), True),
        StructField("UltimaCampanha", IntegerType(), True),
    ])
)

In [7]:
df2= (
    spark
       .read
       .format("csv")
       .option("header", "true")
       .option("delimiter", ",")
       .load('gs://soulcodeprojeto/marketing_campaign.csv',schema=schema)
)

In [8]:
df2.show()

22/03/18 21:29:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

22/03/18 21:29:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, Year_Birth, Education, Marital_Status, Income, Kidhome, Teenhome, Dt_Customer, Recency, MntWines, MntFruits, MntMeatProducts, MntFishProducts, MntSweetProducts, MntGoldProds, NumDealsPurchases, NumWebPurchases, NumCatalogPurchases, NumStorePurchases, NumWebVisitsMonth, AcceptedCmp3, AcceptedCmp4, AcceptedCmp5, AcceptedCmp1, AcceptedCmp2, Complain, Z_CostContact, Z_Revenue, Response
 Schema: ID, AnoNascimento, GrauInstrução, EstadoCivil, Renda, NumCrianca, NumAdolescente, DataConta, UltimaCompra, QtdVinho, QtdFrutas, QtdCarne, QtdPeixe, QtdDoces, QtdOuro, QtdComprasDesconto, QtdComprasWeb, QtdCompraCatalogo, QtdCompraLoja, NumVisitasSite, AceitouCam3, AceitouCam4, AceitouCam5, AceitouCam1, AceitouCam2, Reclamacao, CustoContato, Receita, UltimaCampanha
Expected: AnoNascimento but found: Year_Birth
CSV file: gs://soulcodeprojeto/marketing_campaign.csv
